In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [12]:
hrefs = list()

#Getting the urls for list of ngos recently registered
url = 'http://www.guidestarindia.org/List_of_NGOs.aspx'
r = requests.get(url)
html = r.text
soup = BeautifulSoup(html,"html5lib")
a_tags = soup.find_all('a')
for link in a_tags:
    thref = link.get('href')
    if 'http://www.guidestarindia.org/Summary.aspx?CCReg' in str(thref):
        hrefs.append(thref)
        
#getting the list of certified ngos        
url = 'http://www.guidestarindia.org/CertifiedNGOs.aspx'
r = requests.get(url)
html = r.text
soup = BeautifulSoup(html,"html5lib")
a_tags = soup.find_all('a')
for link in a_tags:
    thref = link.get('href')
    if 'http://www.guidestarindia.org/Summary.aspx?CCReg' in str(thref):
        hrefs.append(thref)
        
hrefs = list(set(hrefs))

In [ ]:
ngo_data = list()
for index_hrefs in range(0,40):
    url = hrefs[index_hrefs]
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html,"html5lib")
    id_data = soup.findAll(True,{'id':'SectionPlaceHolder1_ctl01_ctpTxtCharityName'})
    name = id_data[0].findAll('span')[0].text
    a_tags = soup.find_all('a')
    for link in a_tags:
        thref = link.get('href')
        if 'mailto:' in str(thref):
            email = thref[7:]
            break
    id_data = soup.findAll(True,{'id':'SectionPlaceHolder1_ctl01_ccAddrState'})
    state = id_data[0].findAll('span')[1].text
    id_data = soup.findAll(True,{'id':'SectionPlaceHolder1_ctl01_DynVariableList2_ctl08_txtPhoneNum'})
    ph2 = id_data[0].findAll('span')[0].text
    try:
        id_data = soup.findAll(True,{'id':'SectionPlaceHolder1_ctl01_DynVariableList2_ctl08_txtPhoneArea'})
        ph1 = id_data[0].findAll('span')[0].text
    except:
        ph1 = ''
    phone = ph1+ph2
    url = url.replace('Summary','Organisation')
    r = requests.get(url)
    html = r.text
    soup = BeautifulSoup(html,"html5lib")
    try:
        all_lists = soup.find_all('ul')
        for l in all_lists:
            for l2 in l.find_all('ul'):
                if len(l2.find_all('a'))==0:
                    category = (l2.find_all('li')[0].text).strip()
    except:
        category = ''
    ngo_data.append({'Name': name,
                     'email':email,
                    'state': state,
                    'phone':phone,
                    'category':category})
ngo_data = pd.DataFrame(ngo_data)
ngo_data.head(30)

In [7]:
ngo_data = pd.DataFrame(ngo_data)
ngo_data.head()

,Name,category,email,phone,state
0,Santipur Computer Institute,Social Services,sci8@rediffmail.com,8926524947,West Bengal
1,SOCIAL AWARNESS AND VOLUNTARY EDUCATION (SAVE)...,Social Services,save-ngo@eth.net,4212428100,Tamil Nadu
2,Jan Gramin Vikas Sansthan,Social Services,jgvsjkn@gmail.com,9450571541,Uttar Pradesh
3,Narsinha Sevabhavi Sanstha,Social Services,jos_dig@rediffmail.com,9890559261,Maharashtra
4,Diya India Foundation,Social Services,diya@diyaindia.org,1147041898,Delhi
